In [14]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [15]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [16]:
%run -i ../../zkstats/core.py

In [17]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

In [18]:
def mode_within(data_array, percent):
    max_sum_freq = 0
    mode = data_array[0]

    for check_val in set(data_array):
        sum_freq = sum(1 for ele in data_array if abs(ele - check_val) <= abs(percent * check_val / 100))

        if sum_freq > max_sum_freq:
            mode = check_val
            max_sum_freq = sum_freq

    return mode

In [24]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

f_raw_input = open(data_path, "r")
data = np.array(json.loads(f_raw_input.read())["input_data"][0])
# data_tensor = torch.reshape(torch.tensor(data),(1, len(data), 1))

#  dummy data for data consumer: arbitraryyy, just to make sure after filtered, it's not empty
dummy_data = np.random.uniform(1,100, len(data))
json.dump({"input_data":[dummy_data.tolist()]}, open(dummy_data_path, 'w'))

# where(element > 30)
# dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 ))
# gt30_dummy_data_tensor = dummy_data_tensor[dummy_data_tensor > 30].reshape(1,-1,1)
dummy_theory_output = torch.tensor(mode_within(dummy_data[dummy_data>30],1))





In [25]:
# Verifier/ data consumer side: send desired calculation
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)

    def forward(self,X):
        # where part
        min_X = torch.min(X)
        fil_X = torch.tensor([ele[0] if ele[0]>30 else min_X -1  for ele in X[0]])
        count_equal = torch.sum((torch.abs(X-self.w)<=torch.abs(0.01*self.w)).double())
        result = torch.tensor([torch.logical_or(torch.sum((torch.abs(X-ele[0])<=torch.abs(0.01*ele[0])).double())<=count_equal, torch.abs(min_X-1-ele[0])<=torch.abs(0.01*ele[0])) for ele in X[0]])
        return (torch.sum(result) == X.size()[1], self.w)
verifier_define_calculation(verifier_model, verifier_model_path, [dummy_data_path])

/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_89165/3199990219.py:11: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  fil_X = torch.tensor([ele[0] if ele[0]>30 else min_X -1  for ele in X[0]])
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_89165/3199990219.py:11: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  fil_X = torch.tensor([ele[0] if ele[0]>30 else min_X -1  for ele in X[0]])
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_89165/3199990219.py:11: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this war

In [26]:
# prover calculates settings, send to verifier
theory_output = torch.tensor(mode_within(data[data>30],1))


print("Theory_output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)

    def forward(self,X):
        # where part
        min_X = torch.min(X)
        fil_X = torch.tensor([ele[0] if ele[0]>30 else min_X -1  for ele in X[0]])
        count_equal = torch.sum((torch.abs(X-self.w)<=torch.abs(0.01*self.w)).double())
        result = torch.tensor([torch.logical_or(torch.sum((torch.abs(X-ele[0])<=torch.abs(0.01*ele[0])).double())<=count_equal, torch.abs(min_X-1-ele[0])<=torch.abs(0.01*ele[0])) for ele in X[0]])
        return (torch.sum(result) == X.size()[1], self.w)

prover_gen_settings([data_path], comb_data_path, prover_model,prover_model_path, [2], "resources", settings_path)

/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_89165/461458.py:15: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  fil_X = torch.tensor([ele[0] if ele[0]>30 else min_X -1  for ele in X[0]])
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_89165/461458.py:15: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  fil_X = torch.tensor([ele[0] if ele[0]>30 else min_X -1  for ele in X[0]])
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_89165/461458.py:15: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you 

Theory_output:  tensor(40., dtype=torch.float64)
==== Generate & Calibrate Setting ====
scale:  [2]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":2,"param_scale":2,"scale_rebase_multiplier":10,"lookup_range":[0,0],"logrows":14,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":299,"total_const_size":0,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,2],"model_input_scales":[2],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":[],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_89165/461458.py:17: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  result = torch.tensor([torch.logical_or(torch.sum((torch.abs(X-ele[0])<=torch.abs(0.01*ele[0])).double())<=count_equal, torch.abs(min_X-1-ele[0])<=torch.abs(0.01*ele[0])) for ele in X[0]])


In [27]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0
spawning module 2
spawning module 0


==== setting up ezkl ====


spawning module 2
spawning module 0


Time setup: 1.3078458309173584 seconds
Theory output:  tensor(40., dtype=torch.float64)
!@# compiled_model exists? True
!@# compiled_model exists? True
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 40.0
==== Generating Proof ====


spawning module 2


proof:  {'instances': [[[1780239215148830498, 13236513277824664467, 10913529727158264423, 131860697733488968], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [12341676197686541490, 2627393525778350065, 16625494184434727973, 1478518078215075360]]], 'proof': '1f9ee4c186f5c5a89f93d5ebd603104cd3aef47605542790c6eafdfbf0db47da12f265474542cb1e6d6844469f1fb9270d2a4efd3fbc137507274f89528317ee1ab9e9f0d1cea2fc89eaa4edff89d5f99938a9c7511d268653b0be77f14b348e3057747663dd3e49c2e89ccda09e624d4706b63e3852f5ce019d9c1b22eb47e91d871570117db109dae096aca44d606f3dde8f8c8368234be799da23d10cea961a1996dda8b139b97dd54454cdf41d5e3d4fcda307a2528d4f52445495b1e7d505958e7035a63538b4171a43774a8127d6e271533d8b5fb9bd3c75745147e2872abb221b6e5d925b18377260471e3bc7e7b3ce09bc3bcde57dce119012d2c541037b9919204bdb6537b6bd485155cb973964f1261a485d54d429d9f15fc24bbf2ece8bb42379e104bb699035b839256c880316cdc771714744613fdb8159a59a0916897b752f61ef906f9bb893c804e530b9ee51bb05b79f95fabef46c0f1e

In [29]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path)

num_inputs:  1
prf instances:  [[[1780239215148830498, 13236513277824664467, 10913529727158264423, 131860697733488968], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [12341676197686541490, 2627393525778350065, 16625494184434727973, 1478518078215075360]]]
proof boolean:  1.0
proof result 1 : 40.0
verified
